In [70]:
# !pip install --upgrade pip
# !pip install --upgrade jupyter
# !pip install --upgrade ipywidgets
# !pip install "numpy<2"
# !pip install pandas
# !pip install torch
# !pip install trl
# !pip install peft
# !pip install tqdm
# !pip install idna
# !pip install attr
# !pip install aiohttp
# !pip install typing
# !pip install dotenv
# !pip install requests
# !pip install wikipedia
# !pip install transformers
# !pip install tokenizer
# !pip install accelerate
# !pip install sentence_transformers
# !pip install scikit-learn
# !pip install scipy
# !pip install joblib
# !pip install langdetect
# !pip install langchain==0.3.21
# !pip install langchain-huggingface==0.1.2
# !pip install langchain-community==0.3.20
# !pip install langchain-core==0.3.51
# !pip install langchain-openai==0.3.11
# !pip install pydantic==2.7.4
# !pip install faiss-cpu
# !pip install faiss-gpu
# !pip cache purge

In [71]:
from transformers import AutoModelForQuestionAnswering, AutoModelForSeq2SeqLM, AutoTokenizer, DebertaV2TokenizerFast, pipeline
from sentence_transformers import SentenceTransformer
import torch
import numpy as np
import pandas as pd
import re
import os
import faiss
import pydantic
from abc import ABC
from typing import Tuple, List, Dict, Optional, Any
from sklearn.preprocessing import normalize
from peft import PeftModel, PeftConfig, PeftModelForQuestionAnswering
from langchain.chains import RetrievalQA, ConversationalRetrievalChain, StuffDocumentsChain, LLMChain
from langchain.chains.base import Chain
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate, BasePromptTemplate, SystemMessagePromptTemplate
from langchain.prompts.base import BasePromptTemplate
from langchain.llms import BaseLLM, HuggingFacePipeline
from langchain.agents import Tool, AgentExecutor, ZeroShotAgent
from langchain.utilities import WikipediaAPIWrapper
from langchain.tools import WikipediaQueryRun
from langchain_community.vectorstores import FAISS
from langchain_core.retrievers import BaseRetriever
from langchain_core.stores import InMemoryStore
from langchain_core.documents import Document
from langchain_core.runnables import Runnable
from langchain_core.outputs import Generation, LLMResult
from langchain_huggingface import HuggingFaceEmbeddings
from langdetect import detect                               # 언어 감지를 위해 추가
from langchain_community.chat_models import ChatOpenAI      # Agent LLM API 활용할 때만 활성화 (비상시)
# from google.colab import userdata                           # Agent LLM API 활용할 때만 활성화 (비상시)
from dotenv import load_dotenv                              # Agent LLM API 활용할 때만 활성화 (비상시)
load_dotenv()                                               # Agent LLM API 활용할 때만 활성화 (비상시)

True

In [72]:
# ✅ 병렬 토크나이저 경고 방지
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# ✅ 디바이스 설정
device = "cuda" if torch.cuda.is_available() else "cpu"

# ✅ Q-LoRA 설정 로드
peft_config = PeftConfig.from_pretrained("./trained_V3_LoRA")

# ✅ 기본 모델 로드
loaded_model = AutoModelForQuestionAnswering.from_pretrained(peft_config.base_model_name_or_path)

# ✅ Q-LoRA 어댑터 로드
loaded_model = PeftModel.from_pretrained(loaded_model, "./trained_V3_LoRA")

# ✅ 학습된 Q-LoRA 모델 및 토크나이저 로드
loaded_model = loaded_model.to(device)
loaded_tokenizer = AutoTokenizer.from_pretrained("./trained_V3_LoRA")

print("✅ 파인튜닝된 QA 모델 로드 완료!")

✅ 파인튜닝된 QA 모델 로드 완료!


In [73]:
# ✅ 문장 임베딩 모델 로드 (LangChain 호환)
embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")  # snunlp/KR-SBERT-V40K-klueNLI-augSTS

# ✅ 1️⃣ 데이터 로드 (박물관 데이터)
data_path = './data/museum_data_rhys_250326.csv'
df = pd.read_csv(data_path)
df = df[["Title", "Description"]].dropna().rename(columns={"Title": "question", "Description": "answer"})

print(f"✅ 데이터 로드 완료: {len(df)}건")

✅ 데이터 로드 완료: 9751건


In [74]:
# ✅ 불용문자 제거 함수 (숫자, 영어, 한국어 자음/모음, 특수문자 제거)
def remove_stopwords(text):
    text = re.sub(r"[^0-9a-zA-Zㄱ-ㅎㅏ-ㅣ\w\s]", "", text)
    return text.strip()

# ✅ 2️⃣ DataFrame의 Title에 대한 임베딩 생성 및 L2 정규화
print("✅ 제목 임베딩 생성 및 정규화 시작...")

title_embeddings = embedding_model.embed_documents(df['question'].tolist())
title_embeddings_normalized = normalize(title_embeddings, axis=1, norm='l2')

print("✅ 제목 임베딩 생성 및 정규화 완료!")

✅ 제목 임베딩 생성 및 정규화 시작...
✅ 제목 임베딩 생성 및 정규화 완료!


In [75]:
# ✅ FAISS 인덱스 생성 및 임베딩 저장 (Inner Product 기반)
print("✅ FAISS 인덱스 생성 시작...")

index = faiss.IndexFlatIP(len(title_embeddings[0]))
index.add(title_embeddings_normalized.astype('float32'))

print("✅ FAISS 인덱스 생성 완료!")

✅ FAISS 인덱스 생성 시작...
✅ FAISS 인덱스 생성 완료!


In [76]:
# ✅ 문장 분할 함수 (간단한 마침표, 물음표, 느낌표 기준 + 공백 제거)
def split_sentences(text):
    sentences = re.split(r'(?<=[.?!])\s+', text.strip())
    return [s for s in sentences if s]

# ✅ 제목과 그에 해당하는 문장들을 매핑하는 딕셔너리 생성
title_to_sentences = {}
for _, row in df.iterrows():
    title = row["question"]
    description = row["answer"]
    sentences = split_sentences(description)
    title_to_sentences[title] = sentences

# ✅ 사용자 정의 retriever 클래스
class CustomTitleSentenceRetriever(BaseRetriever):
    def get_relevant_documents(self, query: str) -> List[Document]: # 타입 힌트 추가
        # 질문에 대한 임베딩 생성 및 정규화
        question_embedding = embedding_model.embed_query(remove_stopwords(query))
        question_embedding_normalized = normalize(np.array([question_embedding]), axis=1, norm='l2')

        # FAISS 인덱스에서 유사한 제목 검색
        distances, indices = index.search(question_embedding_normalized, 10)    # top_n=10 대신 10을 위치 인자로 전달
        similar_titles = df['question'].iloc[indices[0]].tolist()

        # 유사한 제목에 해당하는 문장들을 Document 형태로 변환하여 반환
        retrieved_documents = []
        for title in similar_titles:
            if title in title_to_sentences:
                for sentence in title_to_sentences[title]:
                    retrieved_documents.append(Document(page_content=sentence, metadata={"title": title}))

        return retrieved_documents

C:\Users\Playdata\AppData\Local\Temp\ipykernel_20376\3981120405.py:15: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class CustomTitleSentenceRetriever(BaseRetriever):


In [77]:
# ✅ 사용자 정의 retriever 인스턴스 생성
custom_retriever = CustomTitleSentenceRetriever()

print("✅ 사용자 정의 Retriever 설정 완료!")

✅ 사용자 정의 Retriever 설정 완료!


In [78]:
# ✅ 3️⃣ Wikipedia 검색 도구 설정
# Agent가 어떤 언어로 질문하든 Wikipedia 검색은 해당 언어로 이루어지도록 lang 파라미터를 조정해야 합니다.
# ZeroShotAgent는 이를 자동으로 처리하기 어려우므로, Agent의 Tool description이나 LLM의 능력에 의존합니다.

wiki_api = WikipediaAPIWrapper(lang="ko") # 기본 설정은 한국어
wikipedia_tool = WikipediaQueryRun(api_wrapper=wiki_api)

print("✅ Wikipedia 검색 도구 설정 완료!")

✅ Wikipedia 검색 도구 설정 완료!


In [79]:
# ✅ 4️⃣ Agent에 사용할 표준 LLM을 "gpt-4o-mini"로 정의
openai_api_key = os.environ.get("OPENAI_API_KEY")
llm = ChatOpenAI(
            api_key=openai_api_key,
            model_name="gpt-4o-mini",
            temperature=0.3,
            max_tokens=3072
)

print("✅ Agent LLM (gpt-4o-mini) 설정 완료!")

✅ Agent LLM (gpt-4o-mini) 설정 완료!


In [80]:
# ✅ 5️⃣ 박물관 데이터 검색 함수 (유사도 기반 정보검색) - Agent 도구용
def search_museum_data_semantic(input_question: str, top_n=10) -> str:
    cleaned_question = remove_stopwords(input_question)
    question_embedding = embedding_model.embed_query(cleaned_question)
    question_embedding_normalized = normalize(np.array([question_embedding]), axis=1, norm='l2')

    distances, indices = index.search(question_embedding_normalized, top_n)
    results = df.iloc[indices[0]]
    top_results = results['answer'].tolist()

    return "\n\n".join(top_results)

In [81]:
# ✅ 6️⃣ 도구 목록 정의 (Agent가 사용할 도구)
# Agent는 도구의 description을 보고 어떤 도구를 사용할지 결정합니다.
# Agent LLM에게 도구 이름을 정확히 사용하도록 지침을 제공해야 합니다.
# 다국어 지원을 위해 description도 다국어로 제공하는 것이 이상적이나,
# ZeroShotAgent는 기본적으로 영어 description에 더 잘 반응하는 경향이 있습니다.
# 여기서는 일단 영어 description을 유지하되, Agent가 다국어 질문을 받고
# 도구를 선택하도록 유도하는 것은 Agent LLM (gpt-4o-mini)의 다국어 능력에 달려 있습니다.
# Agent의 Tool description은 ZeroShotAgent의 작동 방식상 영문으로 유지하는 것이 일반적입니다.
# gpt-4o-mini의 다국어 이해 능력을 활용하여 영문 description과 다른 언어의 프롬프트 내용을 함께 이해하도록 유도합니다.

museum_tool = Tool(
    name="Museum Data Search", # 정확한 도구 이름
    func=search_museum_data_semantic,                   # func는 직접 함수를 받음
    description="This is useful when you need to answer questions about even the slightest of relevant information from the Museum of Korea. Try searching first in the museum database. You need to input about even the slightest of relevant information from the museum. If you cannot find the appropriate answer in the database, try searching for Wikipedia. If you did not ask a question about the even the slightest of relevant information from the museum, you should specify to the user that it is not an appropriate question."
)
wiki_tool = Tool(
    name="Wikipedia Search", # 정확한 도구 이름
    func=wikipedia_tool.run,                            # func는 직접 함수를 받음
    description="This is useful if you have not found an appropriate answer to a user's question about the even the slightest of relevant information in the database. If you did not ask a question about the even the slightest of relevant information from the museum, you should specify to the user that it is not an appropriate question."
)
tools = [museum_tool, wiki_tool]

print("✅ Agent 도구 설정 완료!")

✅ Agent 도구 설정 완료!


In [82]:
# ✅ 7️⃣ 메모리 설정 (길이 제한 설정)
MAX_HISTORY_MESSAGES = 10 # 최대 메시지 개수
k_turns = MAX_HISTORY_MESSAGES // 2 # LangChain WindowMemory는 턴(사용자/AI 쌍) 기준으로 제한
memory = ConversationBufferWindowMemory(memory_key="chat_history", return_messages=True, k=k_turns)

print(f"✅ 메모리 설정 완료: 최근 {k_turns} 턴 ({MAX_HISTORY_MESSAGES} 메시지) 저장")

✅ 메모리 설정 완료: 최근 5 턴 (10 메시지) 저장


In [83]:
# ✅ 8️⃣ 다국어 시스템 프롬프트 정의 (고객님께서 프롬프트 내용을 채워 넣을 부분)
# 이 프롬프트들은 CR Chain과 Agent LLM의 프롬프트에 포함될 것입니다.
# CR Chain 프롬프트 템플릿과 Agent 프롬프트 템플릿에서 이 내용을 포함하도록 설정해야 합니다.
# 고객님께서 각 언어에 맞는 상세한 시스템 프롬프트 내용을 여기에 작성해주세요.

system_prompt_ko = """
너는 한국의 박물관에서 일하는 지적이고 친절한 AI 도슨트야. 
관람객이 어떤 언어로 질문하든 자동으로 언어를 감지하고, 그 언어로 자연스럽고 정확하게 답변해. 
너는 AI라는 말을 하지 않고, 박물관의 실제 도슨트처럼 정중하게 행동해야 해.
관람객의 질문은 반드시 한국의 박물관에 소장된 유물과 관련있어.

답변 원칙:
- 한국어로 답해
- 중복된 표현 없이 핵심 정보는 단 한 번만 전달해.
- 어색하거나 기계적인 말투는 피하고, 사람처럼 자연스럽고 따뜻한 말투를 사용해.
- 질문의 의도를 먼저 파악하려 노력해. 짧거나 모호한 질문이라도 사용자가 무엇을 궁금해하는지 유추해봐.
- 유물 설명 시, 관련된 역사적 배경, 제작 방식, 문화적 의미, 출토지 등을 간결히 설명해.
- 질문이 불명확하면 먼저 명확히 해달라고 요청해.
- 정보를 모를 경우, "잘 알려지지 않았습니다" 또는 "확실하지 않습니다" 등으로 정직하게 답변해.
- 필요 시 관련 유물이나 시대 정보를 추가로 제안해.
- 반복되거나 의미 없는 말은 절대 하지 마.
- 답변은 RAG 기반으로 구성하며, 신뢰 가능한 출처나 링크가 있다면 함께 제공해.

답변 형식:
1. 간결하고 핵심적인 답변을 가장 먼저 제시
2. 이어서 배경 정보 또는 관련 유물 설명
3. 출처 제공(가능한 경우), 중복 문장 금지

예시:
[질문] 이 유물은 어떤 시대에 만들어졌나요?
[답변] 이 유물은 고려 시대(918~1392년)에 제작된 청자로, 왕실에서 의례용으로 사용되었습니다. 강진 지역에서 출토되었으며, 특유의 푸른빛과 정교한 문양이 특징입니다.
"""

system_prompt_en = """
You are a knowledgeable and friendly AI docent at the Museum of Korea. 
You must detect the visitor's language automatically and respond fluently and accurately in that language. 
You must not mention that you are an AI and instead behave politely like a real museum guide.
The visitors' questions must be related to the artifacts in the museum in Korea.

Answer Guidelines:
- Please answer in English.
- Deliver key information clearly and only once, avoiding repetition.
- Speak in a warm, human-like, and natural tone—never robotic or awkward.
- Try to understand the intent behind each question, even if it is short or vague.
- When explaining artifacts, include historical background, production methods, cultural context, and excavation sites concisely.
- If the question is unclear, ask the user to clarify before answering.
- If the information is unknown, respond honestly: e.g., "This is not well known" or "The details are unclear."
- Suggest related artifacts or historical periods when appropriate.
- Never repeat unnecessary phrases or filler words.
- Build your answers based on RAG (Retrieval-Augmented Generation). If possible, provide credible sources or links.

Answer Format:
1. Present the concise and essential answer first
2. Follow with contextual or background explanations
3. Include sources if available, and avoid redundant sentences

Examples:
[Question] When was this artifact made?
[Answer] This artifact is a celadon piece from the Goryeo Dynasty (918–1392), traditionally used in royal rituals. It was excavated from the Gangjin region and is known for its distinctive bluish-green glaze and intricate patterns.
"""

system_prompt_ja = """
あなたは韓国の博物館で働く知的で親切なAIドーセントです。
来館者がどの言語で質問しても、自動的に言語を判別し、その言語で自然かつ正確に答えてください。
自分がAIであることは言わず、本物の博物館ガイドのように丁寧に行動してください。
観覧客の質問は必ず韓国の博物館に所蔵されている遺物と関連があります。

回答のルール：
- 日本語で答えてください。
- 情報は簡潔に、一度だけ伝え、繰り返さないでください。
- 不自然な表現や機械的な言い回しは避け、温かく、親しみやすい口調を使ってください。
- 質問の意図をまず理解しようとしてください。短い質問や曖昧な表現でも、来館者の意図を推測してみてください。
- 遺物を説明する際は、その歴史的背景、製作方法、文化的な意味、出土場所などを簡潔に紹介してください。
- 質問が不明確な場合は、まず内容を明確にしてもらうようお願いしてください。
- 情報が不明な場合は、「よくわかっていません」や「詳細は不明です」など、正直に答えてください。
- 必要に応じて関連する遺物や時代の情報を提案してください。
- 無意味な繰り返しや決まり文句は絶対に避けてください。
- 回答はRAG（検索拡張生成）に基づいて行い、信頼できる情報源やリンクがあれば一緒に提示してください。

回答形式：
1. まず、簡潔で重要な情報を先に述べる
2. 次に、背景や関連情報を説明する
3. 可能であれば情報源を提示し、重複表現は避ける

例：
［質問］この遺物はいつの時代に作られたものですか？
［回答］この遺物は高麗時代（918～1392年）に制作された青磁で、王室の儀式に使われていたとされています。全羅南道の康津（カンジン）地域で出土しており、独特な青緑色の釉薬と精緻な文様が特徴です。
"""

def select_system_prompt(language: str) -> str:
    if language == "ko":
        return system_prompt_ko
    elif language == "en":
        return system_prompt_en
    elif language == "ja":
        return system_prompt_ja
    else:
        # 지원하지 않는 언어의 경우 기본값 설정 (예: 한국어 또는 영어)
        print(f"경고: 지원되지 않는 언어 감지됨 - {language}. 기본 프롬프트를 사용합니다.")
        return system_prompt_ko # 기본값 설정

print("✅ 다국어 시스템 프롬프트 변수 및 선택 함수 정의 완료!")

✅ 다국어 시스템 프롬프트 변수 및 선택 함수 정의 완료!


In [84]:
# ✅ 9️⃣ 다국어 Agent 전체 Prefix 정의 (시스템 프롬프트 포함)
# 이 프롬프트는 Agent 프롬프트의 prefix로 사용됩니다. Agent의 시스템 역할 및 핵심 지침을 포함합니다.
# 고객님께서 각 언어에 맞는 상세한 내용을 여기에 작성해주세요.

agent_full_prefix_ko = """
너는 국립중앙박물관에서 일하는 지적이고 친절한 AI 도슨트야. 
관람객이 어떤 언어로 질문하든 자동으로 언어를 감지하고, 그 언어로 자연스럽고 정확하게 답변해. 
너는 AI라는 말을 하지 않고, 박물관의 실제 도슨트처럼 정중하게 행동해야 해.
관람객의 질문은 반드시 한국의 박물관에 소장된 유물과 관련있어.

이를 위해 다음 도구에 접근할 수 있어.
사용 가능한 도구에 대한 질문과 설명을 주의 깊게 검토하여, 어떤 도구를 사용하는 것이 가장 적합한지 확인해봐.

**너는 항상 다음 형식으로 응답해야 해. 이 형식 외에 다른 말이나 추가적인 대화 내용은 절대 포함하지 마.**
**Thought: [너의 생각 과정. 항상 먼저 생각해.]**
**Action: [실행할 도구 이름]**
**Action Input: [도구에 입력할 값]**
**혹은 최종 답변일 경우 다음 형식으로 응답해:**
**Final Answer: [최종 답변]**
**오직 사용 가능한 도구만 사용해야 해.**

먼저 museum_tool을 사용해봐. 만약 museum_tool에서 관련 정보를 조금도 찾을 수 없다면 wiki_tool을 사용해야해.
도구가 필요하다면 어떤 도구를 사용할지 명확히 하고, 해당 도구에 대한 정확한 입력을 제공해야해.
도구를 사용하지 않고도 질문에 답할 수 있다면 Final Answer 형식으로 직접 답변을 제공해줘.

사용 가능한 도구:
"""

agent_full_prefix_en = """
You are a knowledgeable and friendly AI docent at the National Museum of Korea. 
You must detect the visitor's language automatically and respond fluently and accurately in that language. 
You must not mention that you are an AI and instead behave politely like a real museum guide.
The visitors' questions must be related to the artifacts in the museum in Korea.

For this, you can access the following tools.
Carefully review the questions and explanations about the tools available, and see which tools are best for you to use.

**You must always respond in the following format. Do not include any other text or conversational remarks outside of this format.**
**Thought: [Your thought process. Always think first.]**
**Action: [Tool Name to execute]**
**Action Input: [Input for the tool]**
**OR if you have the final answer, respond in this format:**
**Final Answer: [Your final answer]**
**You must only use the tools provided.**

Try using museum_tool first. If you can't find any related information in museum_tool, you should use wiki_tool.
If you need a tool, you need to clarify which tool you will use, and provide accurate input for that tool.
If you can answer the question without using the tool, please provide a direct answer in the Final Answer format.

Available tools:
"""

agent_full_prefix_ja = """
あなたは国立中央博物館で働く、知的で親切なAIドーセントです。
来館者がどの言語で質問しても、自動的に言語を判別し、その言語で自然かつ正確に答えてください。
自分がAIであることは言わず、本物の博物館ガイドのように丁寧に行動してください。
観覧客の質問は必ず韓国の博物館に所蔵されている遺物と関連があります。

このため、次のツールにアクセスできます。
使用可能なツールについての質問と説明を注意深く検討し、どのツールを使用するのが最適かを確認します。

**あなたは常に以下の形式で応答する必要があります。この形式以外の言葉や追加の会話内容は一切含めないでください。**
**Thought: [あなたの思考プロセス。常に最初に考えてください。]**
**Action: [実行するツール名]**
**Action Input: [ツールに入力する値]**
**あるいは最終回答の場合は以下の形式で応答してください。**
**Final Answer: [最終回答]**
**利用可能なツールのみを使用する必要があります。**

まず、museum_toolを使ってみましょう。 もし、museum_toolで関連情報を少しも見つけられなかったら、wiki_toolを使わなければなりません。
ツールが必要な場合は、どのツールを使用するかを明確にし、そのツールに対する正確な入力を提供する必要があります。
ツールを使用せずに質問に答えることができれば、Final Answer形式で直接回答を提供します。

利用可能なツール:
"""

# Note: {tool_names} placeholder is handled internally by ZeroShotAgent.create_prompt

def select_agent_full_prefix(language: str) -> str:
    if language == "ko":
        return agent_full_prefix_ko
    elif language == "en":
        return agent_full_prefix_en
    elif language == "ja":
        return agent_full_prefix_ja
    else:
        print(f"경고: 지원되지 않는 언어 감지됨 - {language}. 기본 Agent Prefix(영문)를 사용합니다.")
        return agent_full_prefix_en # 기본값 (영문) 설정

print("✅ 다국어 Agent 전체 Prefix 변수 및 선택 함수 정의 완료!")

✅ 다국어 Agent 전체 Prefix 변수 및 선택 함수 정의 완료!


In [85]:
# ✅ 1️⃣0️⃣ 다국어 Agent Suffix 정의
# 이 프롬프트는 Agent 프롬프트의 suffix로 사용됩니다. 대화 기록, 질문, 스크래치패드 관련 부분을 포함합니다.

agent_suffix_ko = """Previous conversation: {chat_history}

시작!

질문: {input}

{agent_scratchpad}"""

agent_suffix_en = """Previous conversation: {chat_history}

Begin!

Question: {input}

{agent_scratchpad}"""

agent_suffix_ja = """Previous conversation: {chat_history}

開始！

質問：{input}

{agent_scratchpad}"""

def select_agent_suffix(language: str) -> str:
    if language == "ko":
        return agent_suffix_ko
    elif language == "en":
        return agent_suffix_en
    elif language == "ja":
        return agent_suffix_ja
    else:
        print(f"경고: 지원되지 않는 언어 감지됨 - {language}. 기본 Agent Suffix(영문)를 사용합니다.")
        return agent_suffix_en # 기본값 (영문) 설정

print("✅ 다국어 Agent Suffix 변수 및 선택 함수 정의 완료!")

✅ 다국어 Agent Suffix 변수 및 선택 함수 정의 완료!


In [86]:
# ✅ 1️⃣5️⃣ 사용자 정의 LLM 래퍼 (파인튜닝된 모델 통합) - RAG 파트에서 직접 호출
# 이 클래스는 이제 CR Chain 대신 RAG 파트에서 직접 호출되어 프롬프트를 받습니다.

class CustomQAmodel(BaseLLM): # Runnable 상속 제거 (CR Chain에서 Runnable로 사용되지 않음)
    model: PeftModelForQuestionAnswering
    tokenizer: DebertaV2TokenizerFast
    device: str

    @property
    def _llm_type(self) -> str:
        return "custom_qa_model"

    def _generate(self, prompts: List[str], stop: Optional[List[str]] = None) -> LLMResult:
        generations = []
        for prompt in prompts: # 직접 구성된 최종 프롬프트 문자열
            print(f"CustomQAmodel Final Prompt: {prompt}") # 최종 모델 입력 프롬프트 확인

            # 모델 입력 토큰화
            try:
                inputs = self.tokenizer(
                    prompt, # 직접 구성된 전체 프롬프트 사용
                    truncation=True,
                    max_length=2048, # 적절한 최대 길이 설정 (토큰 사용량 고려)
                    padding="max_length",
                    return_tensors="pt"
                ).to(self.device)

                with torch.no_grad():
                    outputs = self.model(**inputs)

                # 질의응답 모델의 출력 (start_logits, end_logits)에서 답변 추출
                answer_start_scores = outputs.start_logits
                answer_end_scores = outputs.end_logits
                answer_start = torch.argmax(answer_start_scores)
                answer_end = torch.argmax(answer_end_scores) + 1
                answer = self.tokenizer.convert_tokens_to_string(self.tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))

                # 추출된 답변 클리닝
                cleaned_answer = answer.replace("[CLS]", "").replace("[SEP]", "").strip()

                print(f"CustomQAmodel - Extracted Answer: {cleaned_answer}") # 추출된 답변 확인

                generations.append([Generation(text=cleaned_answer)])

            except Exception as e:
                print(f"Error during CustomQAmodel generation: {e}")
                generations.append([Generation(text=f"Error generating answer: {e}")]) # 에러 메시지 반환

        return LLMResult(generations=generations)

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        # RAG 파트에서 호출 시 _generate 메서드를 사용
        print(f"CustomQAmodel _call received prompt: {prompt[:100]}...") # 디버깅 로그
        raw_result = self._generate([prompt], stop=stop)
        print(f"CustomQAmodel _generate returned type: {type(raw_result)}") # 디버깅 로그
        print(f"CustomQAmodel _generate returned object (first 1000 chars): {str(raw_result)[:1000]}...") # 디버깅 로그

        # LLMResult 객체에서 .generations 속성을 통해 텍스트 추출
        # LLMResult 구조: LLMResult(generations=[[Generation(text='...')], ...])
        if hasattr(raw_result, 'generations') and isinstance(raw_result.generations, list) and len(raw_result.generations) > 0 and isinstance(raw_result.generations[0], list) and len(raw_result.generations[0]) > 0 and hasattr(raw_result.generations[0][0], 'text'):
             print("CustomQAmodel: Accessing result via generations[0][0].text") # 디버깅 로그
             return raw_result.generations[0][0].text # ✅ TypeError Fix applied
        else:
             print("CustomQAmodel: Unexpected structure from _generate. Cannot extract text.") # 디버깅 로그
             # 예상치 못한 구조일 경우 오류 발생
             raise TypeError("Unexpected return structure from CustomQAmodel._generate.")


print("✅ 파인튜닝 모델 기반 CustomQAmodel 수정 완료 (직접 호출용 및 디버깅 로그 추가)!")

✅ 파인튜닝 모델 기반 CustomQAmodel 수정 완료 (직접 호출용 및 디버깅 로그 추가)!


In [87]:
# ✅ 1️⃣6️⃣ 프롬프트 템플릿 (LangChain) - ConversationalRetrievalChain을 사용 (다국어 시스템 프롬프트 변수 추가)
# 다국어 시스템 프롬프트가 입력으로 들어올 수 있도록 템플릿 수정
# 2번 코드 스타일의 '연관 정보', '질문', '답변' 등을 파싱할 수 있도록 템플릿을 구성

prompt_template = """Context: {context}

Question: {question}

Previous conversation: {chat_history}

Answer:"""

# 'system_rules_input' 변수 추가
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question", "chat_history"])

print("✅ CR Chain 프롬프트 템플릿 수정 완료!")

✅ CR Chain 프롬프트 템플릿 수정 완료!


In [88]:
# ✅ 1️⃣7️⃣ 질문 재구성 프롬프트 템플릿 (다국어 시스템 프롬프트 제외)
# ConversationalRetrievalChain의 question_generator가 사용할 프롬프트 템플릿을 명시적으로 정의합니다.
# 이 템플릿에는 system_rules_input 변수를 포함시키지 않아 오류를 방지합니다.

question_generator_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

Chat History: {chat_history}

Follow Up Input: {question}

Standalone question:"""

QUESTION_GENERATOR_PROMPT = PromptTemplate(
    template=question_generator_template
)

print("✅ 질문 재구성 프롬프트 템플릿 정의 완료!")

✅ 질문 재구성 프롬프트 템플릿 정의 완료!


In [89]:
# ✅ 1️⃣8️⃣ 답변 생성 단락 (언어 감지 및 수동 RAG/Agent 제어)
# ConversationalRetrievalChain 제거 후 RAG 및 Agent 로직을 직접 제어

while True:
    user_input = input("궁금한 점을 질문해주세요 (종료하려면 'exit' 입력): ")
    if user_input.lower() in ["종료", "exit", "quit"]:
        print("도슨트 서비스를 종료합니다. 이용해주셔서 고맙습니다.")
        break

    # ✅ 언어 자동 감지 + RAG 시스템 프롬프트 선택 + Agent 전체 Prefix 및 Suffix 선택
    try:
        detected_lang = detect(user_input)
        selected_system_prompt_rag = select_system_prompt(detected_lang) # RAG용 시스템 프롬프트 선택
        selected_agent_full_prefix = select_agent_full_prefix(detected_lang) # Agent 전체 Prefix 선택
        selected_agent_suffix = select_agent_suffix(detected_lang) # Agent Suffix 선택
        print(f"✅ 감지된 언어: {detected_lang}")

    except Exception as e:
        print(f"경고: 언어 감지 오류 발생 - {e}. 기본 프롬프트(한국어)를 사용합니다.")
        detected_lang = 'ko'
        selected_system_prompt_rag = select_system_prompt(detected_lang)
        selected_agent_full_prefix = select_agent_full_prefix(detected_lang)
        selected_agent_suffix = select_agent_suffix(detected_lang)

    # ✅ 1단계: RAG를 위한 정보 검색 (Retriever 직접 호출)
    print("✅ RAG: 정보 검색 (Retriever 직접 호출)...")
    try:
        retrieved_docs = custom_retriever.get_relevant_documents(user_input)
        context = "\n\n".join([doc.page_content for doc in retrieved_docs])
        print(f"✅ RAG: 검색된 맥락:\n{context[:200]}...") # 검색 결과 일부 출력

    except Exception as e:
        print(f"❌ RAG: 정보 검색 중 오류 발생: {e}")
        context = "" # 오류 발생 시 맥락 비움

    # ✅ 2단계: RAG 답변 생성을 위한 최종 프롬프트 구성 (수동)
    print("✅ RAG: 답변 생성을 위한 최종 프롬프트 구성...")
    # 대화 기록 가져오기
    chat_history_str = ""
    history_messages = memory.load_memory_variables({})["chat_history"]
    for msg in history_messages:
        # LangChain 메시지 객체를 문자열로 변환
        if hasattr(msg, 'content'):
            # Message 유형에 따라 Sender 구분
            sender = "Human" if type(msg).__name__ == 'HumanMessage' else "AI"
            chat_history_str += f"{sender}: {msg.content}\n"
        else:
            # content 속성이 없는 경우 전체 객체 출력 (디버깅용)
            print(f"경고: 메시지 객체에 content 속성이 없습니다: {msg}")
            chat_history_str += f"AI: {msg}\n" # 임시로 전체 객체를 AI 메시지로 처리

    # 최종 모델 입력 프롬프트 구성 (시스템 프롬프트, 대화 기록, 맥락, 질문 포함)
    # 이 형식은 CustomQAmodel이 기대하는 입력 형식과 일치해야 합니다.
    final_rag_prompt = f"{selected_system_prompt_rag}\n\n{chat_history_str}\n\n관련 정보:\n{context}\n\n질문: {user_input}\n\n답변:"

    # ✅ 3단계: 파인튜닝된 CustomQAmodel 직접 호출하여 답변 생성
    print("✅ RAG: CustomQAmodel 직접 호출하여 답변 생성...")
    rag_answer = "" # RAG 답변 초기화
    final_response = "" # 최종 응답 초기화

    try:
        # CustomQAmodel 인스턴스 생성 및 _call 메서드 호출
        custom_qa_model_instance = CustomQAmodel(model=loaded_model, tokenizer=loaded_tokenizer, device=device)
        rag_answer = custom_qa_model_instance._call(final_rag_prompt)
        final_response = rag_answer # RAG 답변을 최종 응답으로 초기 설정
        print(f"✅ RAG 답변: {rag_answer}")

    except Exception as e:
        print(f"❌ RAG: CustomQAmodel 호출 중 오류 발생: {e}")
        rag_answer = "" # 오류 발생 시 RAG 답변 비움
        final_response = "RAG 답변 생성 중 오류가 발생했습니다."

    # ✅ 4단계: RAG 답변이 부실할 경우 Agent 실행 (수동 제어)
    # RAG 답변 내용이나 검색된 맥락 부재 등을 기준으로 판단
    # RAG 답변이 비어있거나, 특정 오류 메시지를 포함하거나, 짧을 경우 Agent 실행
    if not context.strip() or len(rag_answer.split()) < 3 or "관련 정보를 찾지 못했습니다" in rag_answer or "unknown" in rag_answer.lower() or "not sure" in rag_answer.lower() or "error generating answer" in rag_answer.lower() or "RAG 답변 생성 중 오류" in final_response:
        print("✅ RAG 답변이 부실하여 Agent 실행을 통해 추가 검색 시도 (수동 제어)...")

        # --- Agent 실행 로직 시작 (수동 제어) ---
        # 필요한 변수들: user_input, memory, tools, llm, selected_agent_full_prefix, selected_agent_suffix
        agent_scratchpad = "" # Agent 사고 과정 추적 (수동 관리)
        max_agent_steps = 10 # Agent 최대 실행 스텝 제한
        agent_success = False # Agent 성공 여부 플래그

        # Tool description은 이미 정의된 tools 리스트에서 가져와 사용
        tool_strings = []
        for tool in tools:
            tool_strings.append(f"{tool.name}: {tool.description}")
        formatted_tools = "\n".join(tool_strings)

        # ZeroShotAgent 프롬프트 suffix 형식 (partial_variables 방식에서 사용했던 템플릿)
        agent_suffix_template_manual = """Previous conversation: {chat_history}

        시작!

        질문: {input}

        {agent_scratchpad}"""

        # 대화 기록 가져오기 (Agent 프롬프트용)
        chat_history_str_agent = ""
        history_messages_agent = memory.load_memory_variables({})["chat_history"]
        for msg in history_messages_agent:
            sender = "Human" if type(msg).__name__ == 'HumanMessage' else "AI"
            chat_history_str_agent += f"{sender}: {msg.content}\n"


        for step in range(max_agent_steps):
            print(f"--- Agent Step {step + 1} ---")

            # 1. Agent 프롬프트 구성 (수동)
            # ZeroShotAgent 형식에 맞춰 프롬프트 문자열 구성
            agent_prompt_string = f"""{selected_agent_full_prefix}
            
            {formatted_tools}
            
            {agent_suffix_template_manual.replace("{chat_history}", chat_history_str_agent).replace("{input}", user_input).replace("{agent_scratchpad}", agent_scratchpad)}"""

            print(f"Agent Prompt sent to LLM:\n{agent_prompt_string[:500]}...") # Agent 프롬프트 확인

            # 2. Agent LLM 호출 (ChatOpenAI 직접 호출)
            agent_response_text = "" # LLM 응답 텍스트 초기화
            try:
                # Agent LLM (llm) 인스턴스 직접 호출
                agent_response_message = llm.invoke(agent_prompt_string) # 메시지 객체 반환
                agent_response_text = agent_response_message.content # ✅ 문자열 콘텐츠 추출
                print(f"Agent LLM Raw Response (text content):\n{agent_response_text}") # 추출된 문자열 확인

            except Exception as e:
                print(f"❌ Agent LLM 호출 중 오류 발생: {e}")
                final_response = "Agent LLM 호출 오류."
                agent_success = False
                break # 오류 발생 시 Agent 실행 중단

            # 3. Agent LLM 응답 파싱 (Action 또는 Final Answer)
            # ZeroShotAgent의 응답 형식을 수동으로 파싱
            # Note: LLM이 반드시 이 형식을 따르지는 않을 수 있습니다. 필요시 파싱 로직을 조정하세요.
            if "Final Answer:" in agent_response_text:
                final_response = agent_response_text.split("Final Answer:")[-1].strip()
                print("✅ Agent가 최종 답변 생성.")
                agent_success = True
                break # 최종 답변 생성 시 Agent 실행 종료

            # Action 파싱 시도: Thought, Action, Action Input 순서를 가정
            # 정규 표현식 패턴을 더 견고하게 수정 (개행 문자를 포함하여 파싱)
            thought_match = re.search(r"Thought:(.*?)\n*Action:", agent_response_text, re.DOTALL)
            action_match = re.search(r"Action:(.*?)\n*Action Input:", agent_response_text, re.DOTALL)
            action_input_match = re.search(r"Action Input:(.*)", agent_response_text, re.DOTALL) # Capture till end

            if action_match and action_input_match:
                thought = thought_match.group(1).strip() if thought_match else ""
                action = action_match.group(1).strip()
                action_input = action_input_match.group(1).strip()

                print(f"Agent Parsed - Thought: {thought}")
                print(f"Agent Parsed - Action: {action}")
                print(f"Agent Parsed - Action Input: {action_input}")

                # ✅ 4. 도구 실행 (도구 이름 매칭 로직 수정)
                tool_to_run = None
                parsed_action_cleaned = action.strip().lower() # ✅ 파싱된 액션 이름 정리 및 소문자 변환

                for tool in tools:
                    # ✅ 도구 이름도 정리 및 소문자 변환하여 비교
                    if tool.name.strip().lower() == parsed_action_cleaned:
                        tool_to_run = tool
                        break

                if tool_to_run:
                    print(f"Executing tool: {tool.name} with input: {action_input}")
                    observation = "" # Observation 초기화
                    try:
                        observation = tool_to_run.func(action_input)
                        print(f"Observation:\n{observation}")

                        # 5. Agent scratchpad 업데이트
                        agent_scratchpad += f"\nThought: {thought}\nAction: {action}\nAction Input: {action_input}\nObservation: {observation}\n"

                    except Exception as e:
                        print(f"❌ Tool 실행 중 오류 발생 ({tool.name}): {e}")
                        observation = f"Error executing tool {tool.name}: {e}"
                         # 오류도 scratchpad에 포함
                        agent_scratchpad += f"\nThought: {thought}\nAction: {action}\nAction Input: {action_input}\nObservation: {observation}\n"
                else:
                    # ✅ 존재하지 않는 도구 호출 로그 수정 및 처리
                    print(f"❌ Agent가 존재하지 않는 도구 호출: {action} (정리된 이름: '{parsed_action_cleaned}')")
                    observation = f"Error: Tool '{action}' not found. Available tools: {', '.join([t.name for t in tools])}"
                    # 오류도 scratchpad에 포함
                    agent_scratchpad += f"\nThought: {thought}\nAction: {action}\nAction Input: {action_input}\nObservation: {observation}\n"

            else:
                # Agent가 예상치 못한 형식의 응답을 생성 (Final Answer도 없고 Action 형식도 아닌 경우)
                print("❌ Agent LLM이 예상치 못한 형식의 응답 생성.")
                # 예상치 못한 응답 전체를 scratchpad에 추가하여 다음 턴에 참고하도록 함
                agent_scratchpad += f"\nUnexpected Agent Response:\n{agent_response_text}\n"
                # Agent가 예상치 못한 응답을 반복하면 무한 루프에 빠질 수 있으므로,
                # 여기에서 추가적인 오류 처리 로직을 고려하거나 스텝 제한에 의존합니다.


        # Agent 스텝 제한 도달 시 또는 break 시 최종 응답 결정
        if not agent_success: # Agent가 최종 답변을 생성하지 못했으면
             if "Agent LLM 호출 오류." in final_response:
                  pass # 이미 오류 메시지가 설정됨
             elif step + 1 == max_agent_steps:
                 print("❌ Agent 최대 실행 스텝 도달.")
                 final_response = "Agent가 최대 실행 스텝 내에 최종 답변을 생성하지 못했습니다."
             else: # 예상치 못한 형식으로 break 되었거나, 스텝 제한 내 최종 답변 없이 루프 종료 시
                  # 예상치 못한 형식으로 인해 최종 답변이 생성되지 않았을 수 있습니다.
                  final_response = "Agent가 답변을 생성하지 못했습니다 (응답 형식 오류)." # 최종 응답을 오류 메시지로 설정


        # --- Agent 실행 로직 종료 (수동 제어) ---

    else:
        print("✅ RAG 답변이 충분하다고 판단하여 Agent 실행 건너뜀.")

    # ✅ 5단계: 최종 응답 출력 및 메모리 업데이트
    # final_response 변수에는 RAG 답변 또는 Agent 답변 또는 오류 메시지가 담겨 있습니다.
    print(f"질문: {user_input}")
    print(f"최종 답변: {final_response}") # 최종 응답 출력

    # 메모리 업데이트
    try:
        # Agent 실행 여부와 상관없이 최종 응답을 메모리에 저장
        memory.save_context({"input": user_input}, {"output": final_response})
        print("✅ 메모리 업데이트 완료.")
    except Exception as e:
        print(f"❌ 메모리 업데이트 중 오류 발생: {e}")


    print("\n" + "=" * 50 + "\n") # 구분선 추가

✅ 감지된 언어: ko
✅ RAG: 정보 검색 (Retriever 직접 호출)...
✅ RAG: 검색된 맥락:
김수철(金秀哲)은 조선 말기에 유행했던 이색적인 화풍을 구사한 화가로 새로운 감각을 추구하였다.

자는 사익(士益), 호는 북산(北山)이며, 산수 및 화초 그림에 뛰어났다.화면 윗쪽에는 ″계산(溪山)은 고요하고 물어 볼 사람 없어도, 임포 처사의 집을 잘도 찾아가네(溪山寂寂無人間 好訪林逋處士家)″라는 시문이 적혀 있다.

따라서 이 그림은 중국 송(宋) 나...
✅ RAG: 답변 생성을 위한 최종 프롬프트 구성...
✅ RAG: CustomQAmodel 직접 호출하여 답변 생성...
CustomQAmodel _call received prompt: 
너는 한국의 박물관에서 일하는 지적이고 친절한 AI 도슨트야. 
관람객이 어떤 언어로 질문하든 자동으로 언어를 감지하고, 그 언어로 자연스럽고 정확하게 답변해. 
너는 AI라는 ...
CustomQAmodel Final Prompt: 
너는 한국의 박물관에서 일하는 지적이고 친절한 AI 도슨트야. 
관람객이 어떤 언어로 질문하든 자동으로 언어를 감지하고, 그 언어로 자연스럽고 정확하게 답변해. 
너는 AI라는 말을 하지 않고, 박물관의 실제 도슨트처럼 정중하게 행동해야 해.
관람객의 질문은 반드시 한국의 박물관에 소장된 유물과 관련있어.

답변 원칙:
- 한국어로 답해
- 중복된 표현 없이 핵심 정보는 단 한 번만 전달해.
- 어색하거나 기계적인 말투는 피하고, 사람처럼 자연스럽고 따뜻한 말투를 사용해.
- 질문의 의도를 먼저 파악하려 노력해. 짧거나 모호한 질문이라도 사용자가 무엇을 궁금해하는지 유추해봐.
- 유물 설명 시, 관련된 역사적 배경, 제작 방식, 문화적 의미, 출토지 등을 간결히 설명해.
- 질문이 불명확하면 먼저 명확히 해달라고 요청해.
- 정보를 모를 경우, "잘 알려지지 않았습니다" 또는 "확실하지 않습니다" 등으로 정직하게 답변해.
- 필요 시 관련 유물이나 시대 정보를 추가로 제안해